This notebook explores named entity disambiguation and entity linking to Wikipedia pages. The notebook explores string simlarity and cosine similarity as features in an entity linking model.

In [ ]:
import wikipedia
import spacy
import numpy as np
from collections import Counter
import math

In [ ]:
nlp = spacy.load('en', disable=['parser'])
# workaround if you are getting an error loading the sapcy 'en' module:
# nlp = spacy.load('en_core_web_sm', disable=['parser'])

### Fetch candidate Wiki pages
We will use Wikipedia as our knowledge base. Working with the full Wikipedia database is computationaly intensive and time consuming. In this notebook we will try to correctly link mentions of "Michael Jordan" in natural text to the appropriate entity page on Wikipedia. Let's fetch the 3 most popular Wikipedia pages about Michael Jordan: 
- [Michael Jordan](https://en.wikipedia.org/wiki/Michael_Jordan) (NBA player), 
- [Micahel B. Jordan](https://en.wikipedia.org/wiki/Michael_B._Jordan) (actor), 
- [Michael I. Jordan](https://en.wikipedia.org/wiki/Michael_I._Jordan) (UC Berkeley Professor)<br/>

This will take a few minutes

In [ ]:
jordan_type = {
    'Michael Jordan': 'NBA',
    'Michael B. Jordan': 'actor',
    'Michael I. Jordan': 'professor'
}

page_titles = ['Michael Jordan', 'Michael B. Jordan', 'Michael I. Jordan']

# Fetch each page, store the title and summary text.
pages = []
for title in page_titles:
    print("downloading '{}'".format(title))
    page = wikipedia.page(title)
    pages.append((page.title, page.summary))


### Load Data
While you are loading the Wikipedia pages into your notebook, let's generate some test data to work with! 
Find a sentence with a mention of "Michael Jordan" online. You can use news headlines, exerpts from news articles, blog posts, etc. Please don't use any content from Wikipedia.<br/>
<br/>Open up this [Google sheet](https://docs.google.com/spreadsheets/d/1_IarHbOP4m3DWxjiacFCEIHNOKkSTGdfACO4D8hMqA0/edit#gid=0) and paste your sentence in a new row.

Once everyone has populated the Google sheet, download it as a .csv and save it in `../data/MichaelJordan.csv`. Execute the next cell to load all the test samples.

In [ ]:
def read_data(filename):
    
    """ Read data from file 
    Input: 
        - filename containing one document per line
    Output:
        - a list of spaCy tokenized documents
    """
    
    data=[]
    with open(filename) as file:
        for line in file:
            doc = nlp(line.strip())
            data.append(doc)
    
    print("loaded {} docs".format(len(data)))
    
    return data

docs = read_data("../data/MichaelJordan.csv")


### Q1. String Similarity

We can directly compare the string of the mention to the title of each candidate Wikipedia page. 

TODO: Implement a `string_similarity` function that takes two strings as input and returs a similarity score between 0 and 1, with 1 indicating high similarity and 0 indicating no similarity.

You can choose to meausre word-level or character-level similarity. Come up with your own metric, or look into the common string similarity algorithms: [Hamming distance](https://en.wikipedia.org/wiki/Hamming_distance), [Jaccard index](https://en.wikipedia.org/wiki/Jaccard_index), [Levenshtein distance](https://en.wikipedia.org/wiki/Levenshtein_distance)


In [ ]:
def string_similarity(str1, str2):
    """
    Return a string similarity score betweein 0 and 1, 
    where 1 indicates exact match, 0 indicates no match
    
    e.g. string_similarity('hello world', 'hello world') = 1
    """
    
    # TODO: your code here


Check your string similarity function:

In [ ]:
def check_string_similarity(str1, str2):
    print("'{}' vs. '{}' = {}".format(str1, str2, string_similarity(str1, str2)))
    
check_string_similarity("hello world", "hello world") # should be 1
check_string_similarity("Michael B. Jordan", "Michael I. Jordan")
check_string_similarity("Hillary Clinton", "Bill Clinton")
check_string_similarity("one two three", "four five six") # should be 0

Now we will use the `string_similarity` function to compare entity mentions with Wikipedia page titles to see which Wikipedia page is the best match.

In [ ]:
def rank_string_similarity(docs, pages):
    """
    For each doc return the Wiki page that is the best match based on string similarity
    
    docs - is a list of documents containing entity mentions that we want to Wikify
    pages - is a list of Wikipedia (title, summary) tuples
    
    Returns
    result - a list of (title, score) tuples representing the best Wiki page match for each doc.
    (title is the best Wikipedia page title; score is is the string similarity score)
    """
    
    result = []
    for doc in docs:
        best_score = -1
        best_page = pages[0]
        
        # find the best matching entity in doc
        for e in doc.ents:
            # for each entity calculate a strings similarity score with each Wiki page
            for page in pages:
                sim = string_similarity(e.text, page[0])
                # if new best score - store
                if sim > best_score:
                    best_score = sim
                    best_page = page
        
        result.append((best_page[0], best_score))
    return result
    
def display_result(closest, docs):
    
    """ Print results in readable format"""

    for i in range(len(docs)):
        print(docs[i])
        title = closest[i][0]
        print("{} - {} ({})".format(title, jordan_type[title], closest[i][1]))
        print()      

Execute the cell below to see how well string similarity performs for Michael Jordan entity linking. Observe that string similarity on its own is not enough for correct entity disambiguation.

In [ ]:
string_rank = rank_string_similarity(docs, pages)
display_result(string_rank, docs)


### Q2. Cosine Similarity
Now we will implement a more robust measure of similarity between enity mentions and Wikipedia pages by incorporating the context of each mention

We will use Glove word embeddings to generate vector representations of entity context and Wikipedia pages. Lets load them here. 

You should already have the glove file in your data/ directory. If you get a 'file not found' error, download the top 50K words in the "Common Crawl (42B)"  vectors (300-dimensional) here: [glove.42B.300d.50K.txt](https://drive.google.com/file/d/1n1jt0UIdI3CD26cY1EIeks39XH5S8O8M/view?usp=sharing) and place  in your `data` directory. Skip ahead to the next cell to reformat and load the embeddings.

In [ ]:
def load_embeddings(filename, max_vocab_size):
    """
    Load embeddings from file
    Returns
    - embeddings: a list of embedding vectors
    - vocab: a dictionary of {word: idx} pairs where idx points to the index of the 'word' embedding in the embeddings list
    """
    
    vocab={}
    embeddings=[]
    with open(filename) as file:
        
        cols=file.readline().split(" ")
        num_words=int(cols[0])
        size=int(cols[1])
        embeddings.append(np.zeros(size))  # 0 = padding
        embeddings.append(np.zeros(size))  # 1 = UNK,
        vocab["_0_"]=0
        vocab["_UNK_"]=1
        
        for idx,line in enumerate(file):

            if idx+2 >= max_vocab_size:
                break

            cols=line.rstrip().split(" ")
            val=np.array(cols[1:])
            word=cols[0]
            
            embeddings.append(val)
            vocab[word]=idx+2

    return np.array(embeddings), vocab

embeddings, vocab=load_embeddings("../data/glove.42B.300d.50K.w2v.txt", 50000)

If you just downloaded glove embeddings for the first time, run the cell below. Else, skip ahead

In [ ]:
"""
Run after downloading glove.42B.300d.50K.txt  for the first time
Only run this cell if you got a 'file not found' error above
"""
from gensim.models import Word2Vec, KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

# First we have to convert the Glove format into w2v format; this creates a new file
glove_file="../data/glove.42B.300d.50K.txt"
glove_in_w2v_format="../data/glove.42B.300d.50K.w2v.txt"
_ = glove2word2vec(glove_file, glove_in_w2v_format)

# now load the embeddings
embeddings, vocab=load_embeddings("../data/glove.42B.300d.50K.w2v.txt", 50000)

TODO: Implement the `get_doc_representation` function below to return an embedding vector that represents all words in the input document.

In [ ]:
def get_doc_representation(doc, vocab, embeddings):
    """
    Return one vector that represents the entire array of input tokens.
    Output shape should == embeddings.shape[1]

    Input: 
        - doc: a scpaCy document instance
        - vocab: a dictionary of (word, index) pairs. 
        'index' denotes the location of each word in the 'embeddings' list
        - embeddings: a list of word embeddings
    """
    
    # TODO: your code here


In [ ]:
def cos_similarity(v1, v2):
    """ Returns cosine similarity between two vectors """
    cos = np.dot(v1, v2) / (np.sqrt(np.dot(v1,v1)) * np.sqrt(np.dot(v2,v2)))
    return cos

def rank_cos_similarity(docs, pages, vocab, embeddings):
    """
    For each doc return the Wiki page that is the best match based on cosine similarity
    
    Input:
        - docs: a list of documents containing entity mentions that we want to Wikify
        - pages: a list of Wikipedia (title, summary) tuples
    
    Returns
        - a list of (title, score) tuples representing the best Wiki page match for each doc.
        (title is the best Wikipedia page title; score is is the string similarity score)
    """
    
    result = []
    
    # get page representations
    page_representations = []
    for p in pages:
        summary = p[1]
        emb = get_doc_representation(summary.split(" "), vocab, embeddings)
        page_representations.append((p[0], emb))
    
    for doc in docs:     
        doc_representation = get_doc_representation([token.text for token in doc], vocab, embeddings)
        
        scores = []
        
        for p in page_representations:
            title = p[0]
            emb = p[1]
            sim = cos_similarity(doc_representation, emb)
            scores.append((p[0], sim))

        scores = sorted(scores, key=lambda x: x[1], reverse=True)
    
        result.append(scores[0])
    return result

    

Execute the cell below to see how well cosine similarity performs for Michael Jordan entity linking.

In [ ]:
closest_cos = rank_cos_similarity(docs, pages, vocab, embeddings)
display_result(closest_cos, docs)

### Q3. Other Features?

Based on your results above, can you think of other features that might help improve entity linking performance further? Name one feature and justify why you think it is a good feature.

[your response to Q3 here]